In [ ]:
import math
import numpy as np

class projectilesolver(object):

    def __init__(self, lat1, lon1, alt1, lat2, lon2, alt2):
        self.lat1 = lat1
        self.lon1 = lon1
        self.alt1 = alt1  # Altitude in kilometers (e.g., 0.1 = 100m)
        self.lat2 = lat2
        self.lon2 = lon2
        self.alt2 = alt2

    

    ############ calculated Range ##############
    def shooting(self, initial_velocity, angle_of_projection):
        u = initial_velocity  # (km/s)
        g = 9.8 * (pow(10, -3))
        theta = math.radians(angle_of_projection)  # convert degrees to radians if needed
        result = math.sin(2 * theta)

        range1 = (pow(u, 2)) * result / g

        return range1




    ################    Time of FLight   #################

    def time_of_flight(self, initial_velocity, angle_of_projection):
        u = initial_velocity  # (km/s)
        g = 9.8 * (pow(10, -3))  # gravitational constant
        theta = math.radians(angle_of_projection)  # convert degrees to radians if needed
        result = math.sin(theta)

        time = (2 * u * result) / g
        print(f'Time of flight in seconds :: {time}')
        return time
    
    ################  max height        ##############
    def max_height(self, initial_velocity, angle_of_projection):
        u = initial_velocity  # (km/s)
        g = 9.8 * (pow(10, -3))  # gravitational constant
        theta = math.radians(angle_of_projection)  # convert degrees to radians if needed
        result = math.sin(theta)

        max_height = (pow(u, 2)) * (pow(result, 2)) / (2 * g)
        print(f'max_height Km :: {max_height}')
        return max_height
    


    
    ############  trajectory equation ##############
    def trajectory_equation(self, initial_velocity, angle_of_projection):
        """
        Returns the trajectory equation coefficients in the form:
        y = -a*x^2 + b*x (i.e., y = ax^2 + bx + c, with c = 0)
        where:
        - a = g / (2 * u^2 * cos^2(theta))
        - b = tan(theta)
        """
        u = initial_velocity
        g = 9.8 * 1e-3  # km/s^2
        theta = math.radians(angle_of_projection)

        cos_theta = math.cos(theta)
        if cos_theta == 0:
            raise ValueError("cos(theta) is zero, invalid angle")

        a = g / (2 * u ** 2 * cos_theta ** 2)
        b = math.tan(theta)

        print(f"Parabolic Trajectory Equation: y = -({a:.4f})x² + ({b:.4f})x")
        return -a, b  # 'a' is negative in projectile motion
    
    
    ###################  landingpoint Coordinates  #################
    def predict_landing_point(self,lat1_deg, lon1_deg, azimuth_deg, initial_velocity, angle_of_projection):
        R = 6371.0  # Earth's radius in km (average)
        
        # Convert inputs to radians
        lat1 = math.radians(lat1_deg)
        lon1 = math.radians(lon1_deg)
        azimuth = math.radians(azimuth_deg)
        range_km= self.shooting( initial_velocity, angle_of_projection)
        d_div_R = range_km / R

        # Compute destination point
        lat2 = math.asin(math.sin(lat1) * math.cos(d_div_R) +
                        math.cos(lat1) * math.sin(d_div_R) * math.cos(azimuth))

        lon2 = lon1 + math.atan2(math.sin(azimuth) * math.sin(d_div_R) * math.cos(lat1),
                                math.cos(d_div_R) - math.sin(lat1) * math.sin(lat2))

        # Convert back to degrees
        lat2_deg = math.degrees(lat2)
        lon2_deg = math.degrees(lon2)

        return [lat2_deg, lon2_deg]
    
    # distance between two points 
    def haversine(self,lat1, lon1, lat2, lon2):
        R = 6371.0  # Earth radius
        dlat = math.radians(lat2 - lat1)
        dlon = math.radians(lon2 - lon1)

        a = math.sin(dlat / 2)**2 + math.cos(math.radians(lat1)) * \
            math.cos(math.radians(lat2)) * math.sin(dlon / 2)**2
        return R * 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))  # in km
    
    ##########  angle between enemy ,predicted point( landed position ) and my location ,, with my location As centre
    def latlon_to_cartesian(self,lat, lon):
        lat_rad = math.radians(lat)
        lon_rad = math.radians(lon)
        x = math.cos(lat_rad) * math.cos(lon_rad)
        y = math.cos(lat_rad) * math.sin(lon_rad)
        z = math.sin(lat_rad)
        return np.array([x, y, z])

    def angle_between_points(self, azimuth_deg, initial_velocity, angle_of_projection):
          # A = enemy location
        A = self.latlon_to_cartesian(self.lat2, self.lon2)

        # B = your (launcher) location
        B = self.latlon_to_cartesian(self.lat1, self.lon1)

        # C = predicted landing location
        predicted_lat, predicted_lon = self.predict_landing_point(
            self.lat1, self.lon1, azimuth_deg, initial_velocity, angle_of_projection
        )
        C = self.latlon_to_cartesian(predicted_lat, predicted_lon)

        # Vectors from B to A and B to C
        BA = A - B
        BC = C - B

        # Normalize the vectors
        BA /= np.linalg.norm(BA)
        BC /= np.linalg.norm(BC)

        # Compute the angle using dot product
        dot = np.dot(BA, BC)
        dot = np.clip(dot, -1.0, 1.0)  # Numerical safety
        angle_rad = math.acos(dot)
        angle_deg = math.degrees(angle_rad)

        return angle_deg
    
    def closeOrFar(self):
        z=['You are  tooooo close differ by 0.','You are  close but not that close ','not close adjust the angle','far']
        if self.angle_between_points( azimuth_deg, initial_velocity, angle_of_projection) <= 1:
            return z[0]
        elif self.angle_between_points( azimuth_deg, initial_velocity, angle_of_projection) <= 5:
            return z[1]
        elif self.angle_between_points( azimuth_deg, initial_velocity, angle_of_projection) <= 30:
            return z[2]
        else:
            return z[3]


# input parameters :: initial_velocity
#                     angle of projection 
#                     enemy and our location
lat1, lon1, alt1 = 17.5959743, 78.1244778,0.0        # your position :sangareddy
lat2, lon2, alt2 = 17.5847311, 78.121341, 0.00       #  enemyposition:  hyderabad
initial_velocity = 0.9
angle_of_projection = 18.4997893488487
azimuth_deg = 194.89307021706347   # angle with respect to North

solver = projectilesolver(lat1, lon1, alt1, lat2, lon2, alt2)

# Solve and print
range_km = solver.shooting(initial_velocity, angle_of_projection)
print(f"Projectile Range (horizontal): {range_km} km")

solver.time_of_flight(initial_velocity, angle_of_projection)
solver.max_height(initial_velocity, angle_of_projection)
solver.trajectory_equation(initial_velocity, angle_of_projection)

predicted_lat, predicted_lon = solver.predict_landing_point(lat1, lon1, azimuth_deg, initial_velocity, angle_of_projection)
print(f"Predicted landing point: lat = {predicted_lat:.6f}, lon = {predicted_lon:.6f}")

angle = solver.angle_between_points(azimuth_deg, initial_velocity, angle_of_projection)
print(f"Angle at your location between enemy and landing point: {angle:}°")


print(solver.closeOrFar())
